<a href="https://colab.research.google.com/github/JayroMartinez/game_of_life_sim/blob/main/game_of_life_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CLONE OR UPDATE REPO
%%bash
if [ ! -d game_of_life_sim ]; then
  git clone https://github.com/JayroMartinez/game_of_life_sim.git -q
else
  git -C game_of_life_sim pull -q
fi
pip install -q -r game_of_life_sim/requirements.txt

In [ ]:
# ENVIRONMENT SETUP
import os, sys
sys.path.append("/content/game_of_life_sim/src")
os.chdir("/content/game_of_life_sim")
print("Environment ready.")

In [ ]:
# SINGLE ANIMATED SIMULATION
# --size: Board size length --prob: Initial alive-cell probability
%run -i run_sim.py --size 60 --prob 0.25

In [ ]:
# PARALLEL SIMULATION
# --runs: Simulations to run --size: Board size length --prob: Initial alive-cell probability
!python run_sim.py --runs 100 --size 60 --prob 0.33